# Implementación del algoritmo de Metrópolis para el modelo de Ising 2D con dinámica de Kawasaki. 


En este trabajo se ha implementado el modelo de Ising bidimensional sobre una red cuadrada, representada por una matriz donde el índice de fila $x$ corresponde a la dirección vertical y el índice de columna $y$ a la dirección horizontal. Se han impuesto condiciones de contorno periódicas en la dirección $y$ , lo que significa que los extremos izquierdo y derecho de la red están conectados entre sí. En la dirección $x$ se han aplicado condiciones de contorno libres, de modo que los espines en la primera y última fila no tienen vecinos fuera de la red. Además, se ha fijado el valor de todos los espines de la primera fila en un sentido y el de la última en sentido opuesto. 

La dinámica utilizada en la simulación es la dinámica de Kawasaki, que consiste en seleccionar aleatoriamente un par de espines vecinos y proponer su intercambio. Este mecanismo modela la conservación del número de espines positivos y negativos, característica relevante en materiales ferromagnéticos cerrados donde no hay creación ni destrucción de espines, solo desplazamiento. El intercambio se acepta o rechaza según el criterio de Metrópolis, que depende de la variación de energía y la temperatura del sistema.

La magnetización inicial de la red se ha fijado en cero, es decir, el número de espines +1 y -1 es igual al comenzar la simulación. Como la dinámica de Kawasaki solo permite intercambios de espines, la magnetización total permanece constante a lo largo de toda la evolución temporal del sistema.

Por debajo de la temperatura crítica, el sistema tiende a organizarse en dominios: regiones donde los espines están alineados en la misma dirección. Aunque la magnetización global sigue siendo nula (por conservación), se observan grandes agrupaciones de espines del mismo signo, separadas por fronteras de dominio. Este fenómeno refleja la tendencia de los materiales ferromagnéticos a formar regiones ordenadas a bajas temperaturas.


## Transición Ferromagnética-Paramagnética en sólidos
 Los materiales ferromagnéticos presentan momentos magnéticos elementales permanentes orientados de forma aleatoria. Estos momentos crean un campo magnético intenso que alinea los momentos magnéticos de los átomos cercanos, dividiendo el metal en diferente dominios según la
 dirección que los caracterice. Si un campo magnético externo tiende a alinear dichos dominios
 decimos que el material está magnetizado y permanecerá en dicho estado incluso en ausencia del
 mismo. 

 Los materiales paramagnéticos también presentan momentos magnéticos permanentes en sus átomos, pero estos generan un campo magnético débil que no afecta a los átomos cercanos. Como resultado, los espines tienden a orientarse de manera aleatoria debido a la agitación térmica, y la magnetización neta del sistema es prácticamente nula en ausencia de un campo externo. 

En esta práctica comprobaremos que cualquier material ferromagnético se comporta como paramagnético a partir de una temperatura crítica , llamada temperatura de Curie.


## Dinámica del modelo en función de la temperatura: Energía

En el modelo de Ising, la energía total viene dada por la interacción de los espines con sus vecinos más cercanos. El estado de mínima energía se corresponde con la situación en la que todos los espines se encuentran alineados, mientras que el estado de máxima energía ocurre cuando el sentido de los espines se alterna, es decir, los espines son antiparalelos dos a dos.

 En el caso de la dinámica de kawasaki el estado de mínima energía para una configuración inicial aleatoria se alcanzará a temperaturas bajas y consiste en la formación de dominios de espines alineados. Dado que la magnetización no se conserva, la situación en la que todos los espines se alineen no es posible. 
 
 Por otro lado, la máxima energía que puede alcanzar el sistema se corresponde con energía cero, situación en la que los espines se distribuyen de manera aleatoria. El estado de máxima energía se alcanza a temperaturas bastante altas. 

Se han impuesto **condiciones de contorno periódicas** en la dirección horizontal de la red y **condiciones libres** en la dirección vertical, fijando todos los espines de la primera y la última fila a un sentido único: positivo y negativo, respectivamente.




### Cálculo eficiente de la energía

> La energía de una configuración en el modelo de Ising bidimensional se calcula mediante la siguiente expresión:

$E(S) = -\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} s(i, j) \left[ s(i, j+1) + s(i, j-1) + s(i+1, j) + s(i-1, j) \right]$

donde $s(i, j)$ ) representa el valor del espín en la posición ((i, j)) de la red. El factor $\frac{1}{2}$ evita contar dos veces cada interacción entre pares de espines vecinos. 

Para reducir el costo computacional en la simulación, no se recalcula la energía total de la red tras cada intercambio de espines. En su lugar, se calcula únicamente la diferencia de energía entre las dos configuraciones que difieren solo en el signo de dos espines vecinos (tras realizar el intercambio).

Este procedimiento consiste en evaluar la contribución a la energía únicamente de los espines involucrados en el intercambio. Pongamos que los espines intercambiados se sitúan en las posiciones $(x_1, y_1)$ y $(x_2, y_2)$. Para cada uno de estos espines, se consideran únicamente las interacciones con sus vecinos más próximos, excluyendo el espín con el que se ha realizado el intercambio, ya que ese término se anula al calcular la diferencia de energía (dado que el producto de ambos espines cambia de signo pero su contribución neta es la misma antes y después del intercambio).

De este modo, antes del intercambio para el espín en la posición $(x_1, y_1)$ se suman las interacciones con sus tres vecinos restantes, y lo mismo para el espín en $(x_2, y_2)$. Después del intercambio, se calcula la energía local de cada espín de forma análoga pero con los espines  intercambiados. Así, la variación de energía $\Delta E$ se obtiene de forma eficiente, considerando solo los términos relevantes.



En la implementación en C, la energía total de la configuración se calcula una única vez al inicio de la simulación, utilizando la función `calcularEnergiaConfiguracion`. Posteriormente, tras cada intento de intercambio de espines, no se recalcula la energía total de la red, sino que se evalúa únicamente la variación de energía local ($\Delta E$) asociada al intercambio propuesto. Si el intercambio es aceptado según el criterio de Metrópolis, este incremento $\Delta E$ se suma (o resta) a la energía actual.


Este método evita el costoso cálculo de la energía total en cada paso Montecarlo, mejorando notablemente la eficiencia computacional, especialmente para redes grandes. 

## RESULTADOS EXPERIMENTALES VS PREDICCIÓN TEÓRICA

## Calor específico. 

El tamaño finito del sistema implica que no vamos a poder observar
de forma exacta el comportamiento descrito por la teoría, ya que en
ella los resultados se obtienen tras imponer el límite termodinámico, es
decir, el número N de partículas del sistema tiende a ∞.

 Parecería que cuando $N \to \infty$ nos acercamos al límite de Onsager, donde $ C \to \infty$ cuando $T \to T_c$. Sin embargo, esto es difícil de lograr en la práctica. 

Según Onsager, el calor específico muestra una divergencia logarítmica en la temperatura crítica,  es decir, la solución exacta de Onsager desvela que $ C_v\sim - \ln |T - T_c| $ 

A continuación, se muestran los resultados experimentales del calor específico en función de la temperatura. Las medidas se han realizado para 100000 pasos montecarlo y magnetización inicial nula con la mitad de la región con espines positivos y la otra mitad con espines negativos. 


![SUS32](Cv_vs_Temperatura.png)

El rango experimental posible para la 
transición de fase es bastante amplio, entre 2 < T < 3 (sabemos que la temperatura crítica teórica es 2.269 ). En torno a la zona crítica hemos añadido más puntos para observar mejor el comportamiento en la transición de fase.


De acuerdo con el artículo "The Analytical Expressions for a Finite-Size 2D Ising Model", la diferencia entre la temperatura critica experimental con respecto a la temperatura crítica dada por la solución exacta de Onsager crece como $\sqrt{N}$ ($T$ guarda una relación inversa con $\beta$), y el pico de la capacidad calorífica (magnitud extensiva) como $\ln N$. 


$\beta_c = \beta_{\text{ONS}} \left( 1 + \frac{5}{4 \sqrt{N}} \right)$

Observamos  en la gráfica que efectivamente el pico del calor específico se desplaza hacia la derecha (hacia T superiores) conforme N aumenta y es cada vez más pronunciado. Es decir, tal como predice la ecuación anterior, la temperatura crítica experimental se desplaza hacia la derecha a mayor N. 

 Sin embargo, como el calor específico es una magnitud intensiva, no debería crecer con el tamaño del sistema. Esto se trata claramente de un error. 



Hemos calculado mediante la expresión anterior el valor de la temperatura crítica que realmente deberíamos haber obtenido experimentalmente para los valores de  $N$ que hemos impuesto en el código. En la gráfica se observa que los valores experimentales $T$ de la temperatura crítica obtenidos son mayores que los esperados  (muy cercanos a 3 para N grandes). 
| Tamaño de la red ($N$) | $C_v$  | $T$   | $T_c$ esperados  |
|------------------------|--------|-------|---------|
| 32                     | 2.40   | 2.5   | 1.858   |
| 64                     | 4.30   | 2.5   | 1.962   |
| 128                    | 44.85  | 3.0   | 2.04    |

## Curva de magnetización

Se han obtenido las curvas de magnetización para N=32, 64 y 128, 10000 pasos montecarlo y magnetización inicial nula (espines alineados en un sentido en la primera mitad del dominio y en sentido contrario en la segunda mitad).


Se observa que para N=128 la curva se va pareciendo a una función escalón, con una pendiente muy pronunciada en la transición de fase. El rango de valores en el que parece estar contenida la temperatura crítica está más cerca de la temperatura teórica 2.269 que en el caso del las medidas con el calor específico. 

![magnetización](MAGNETIZACIÓN.png)

# Densidad de partículas promedio en la dirección y en función de la temperatura y para distintos valores de N. 
En esta sección se analiza la densidad de espines +1 y -1 por fila en función de la temperatura, para diferentes tamaños de red. La densidad de espines positivos se ha calculado como la suma de espines +1 en una fila dividida entre el número total de espines de esa fila, es decir, 

$\rho_{+} = \frac{\text{número de espines positivos }}{\text{número total de espines en la fila}}$.

 De manera análoga, se calcula la densidad de espines negativos. Es importante destacar que la suma de ambas densidades en cada fila es siempre 1. 

A bajas temperaturas, se observa que la densidad de espines +1 es mucho mayor que la de espines -1. Esto es coherente con el comportamiento del sistema en el régimen ferromagnético, donde la magnetización es máxima y predominan los dominios magnéticos formados por espines alineados del mismo signo . 

A medida que la temperatura aumenta y el sistema se desordena,  las densidades de espines +1 y -1 tienden a igualarse, aproximándose ambas al valor 1/2. A altas temperaturas, los espines se orientan aleatoriamente y la probabilidad de encontrar un espín +1 o -1 en cualquier posición es prácticamente la misma. Por tanto, en el régimen paramagnético, la densidad de espines positivos y negativos por fila converge a 0.5.

La fila sobre la que se ha promediado la densidad corresponde a la posición de N/4 y las medidas se han realizado para 100000 pasos montecarlo. 

![SUS32](Densidad_vs_Temperatura.png)




# Energía media por partícula en función de la temperatura para diferentes valores de N. 

La energía media por partícula $e$ se ha calculado como el valor promedio de la energía total de la configuración dividido entre $2N^2$, es decir:

$e = \frac{\langle E \rangle}{2N^2}$

donde $\langle E \rangle$ es la energía promedio de la configuración y $N$ es la longitud lateral de la red cuadrada. 

En la gráfica se aprecia que la energía se minimiza para temperaturas bajas, y tiende a cero para temperaturas altas. Las medidas se han realizado para 100000 pasos montecarlo. 

![SUS32](Energia_media_vs_Temperatura.png)



## Susceptibilidad magnética 

La ley de Curie-Weiss describe el comportamiento de la susceptibilidad magnética en materiales ferromagnéticos.

$\chi_m = \frac{C}{T - T_C}$

La ecuación predice que la susceptibilidad magnética diverge cuando la temperatura se aproxima a $T_C$ y a partir de dicho punto decrece con la temperatura. 

Este hecho se observa en las siguientes gráficas, realizadas para N=32, 64 y 128. Los errores no se aprecian debido a la pequeña magnitud de las medidas. Se han realizado para 100000 pasos montecarlo y magnetización inicial nula.

![SUS32](susceptibilidad_vs_Temperatura.png)

## Gráficas de coste


El algoritmo de Metrópolis utilizado tiene la limitación de que los estados que genera no son independientes uno del otro, puesto que cada configuración de espines es una modificación de la anterior. Esto introduce correlaciones entre configuraciones sucesivas, especialmente si se toman datos en cada iteración. Para obtener muestras estadísticamente independientes hemos tomado los datos solo cada 100 pasos montecarlo, lo que ha supuesto un mayor coste computacional .


La siguiente gráfica muestra el coste del programa en función del tamaño de la red incluyendo un break en el bucle interno de un paso Montecarlo, y sin incluirlo. 

![break](Tiemposconbreak.png)

El break  mejora la eficiencia del programa porque permite salir del bucle que recorre los intentos de intercambio de espines en un paso de Montecarlo tan pronto como se acepta un intercambio. Esto evita realizar cálculos innecesarios una vez que se ha aceptado el cambio, pasando al siguiente paso Montecarlo.

Así, el coste del programa aumenta según $N^2$ con el tamaño de la red para el caso menos eficiente, mientras incluyendo el break el coste es lineal con $N$.

La siguiente gráfica muestra cómo varía el tiempo de ejecución del programa en función del tamaño de la red (N), dependiendo del nivel de optimización utilizado en la compilación.

![coste](Tiempos_optimización.png)

La opción O1 prioriza la compacidad del código, minimizando el uso de memoria y reduciendo el tamaño del ejecutable, pero acelera menos el tiempo de ejecución que en el caso de O2 y O3.

O1 puede ser útil cuando los recursos son limitados, pero da un peor rendimiento especialmente cuando N es grande, como se observa en la gráfica para N=140 y N=160.

O3 utiliza una optimización más intensa que O2, por lo que resulta útil en tareas que se realizan muchos cálculos. Sin embargo, para tamaños relativamente pequeños (como N = 60), con O3 obtenemos tiempos mayores debido al coste adicional que algunas optimizaciones más complejas del compilador pueden introducir en el programa.
Pero a medida que el tamaño del problema crece, O3 demuestra una mejora notable, superando a las demás opciones en los casos más exigentes (por ejemplo, cuando N = 130 y N=160).

Para valores medios de N, O2 es la mejor elección, ofreciendo un equilibrio entre rapidez y uso de recursos.

## Características de Joel

| Recurso                        | Valor                                      |
|------------------------------- |--------------------------------------------|
| **Modelo de la CPU**           | AMD EPYC 7401P 24-Core Processor           |
| **CPUs disponibles**           | 48 núcleos                                 |
| **Memoria RAM empleada**          | 2,9 GiB                                    |
| **Memoria RAM total**          | 62 GiB                                     |
| **Frecuencia de reloj (máx)**  | 2000 MHz                                   |
| **Frecuencia de reloj (mín)**  | 1200 MHz                                   |
| **Sistema operativo**          | Ubuntu 22.04.5 LTS                         |

La memoria RAM empleada es la memoria usada en el momento por el sistema operativo y los procesos en ejecución.

El número de CPUs del PC usado es 8 núcleos. La RAM instalada es 8,00 GB (7,65 GB usable) y el procesador 11th Gen Intel(R) Core(TM) i7-1165G7.

# Estudio de la convergencia mediante la varianza de la energía

En el código se ha implementado un criterio de convergencia basado en la varianza de la energía para determinar cuándo el sistema ha alcanzado el equilibrio. El procedimiento es el siguiente:

- Durante la simulación, se almacena el valor de la energía en cada paso de Montecarlo.
- Cada cierto número de pasos (definido por la constante `BLOQUE_CONVERGENCIA`), se calcula la media y la varianza de la energía en ese bloque.

- Si la varianza de los valores de la energía en ese bloque es menor que un umbral (`UMBRAL_CONVERGENCIA`) se considera que el sistema ha alcanzado la convergencia.
- En ese momento termina la simulación  usando `break`.

Este método permite identificar cuándo el sistema ha llegado a un estado estacionario, es decir, cuando las fluctuaciones de la energía se han estabilizado y la varianza es mínima, lo que indica que las configuraciones generadas ya son representativas del equilibrio térmico. 

La siguiente gráfica muestra la energía promedio del sistema para cada paso montecarlo para T=1.0. En el código se han impuesto 10000 pasos montecarlo, pero en el gráfica se observa que la convergencia (cuando la energía se estabiliza a un valor mínimo) se alcanza en el paso 1000. 


El umbral de convergencia de la energía ronda en torno a los 20-40 para temperaturas bajas. Se han comparado la dispersión de los valores de la energía respecto a su promedio en bloques de 100 pasos montecarlo. 

![convergencia](convergenciae.png)



## SIMULACIONES
La siguiente simulación corresponde a una red de tamaño N=100 para T=1.0 y 10000 pasos montecarlo. Vemos como el sistema partiendo de una configuración inicialmente desordenada y magnetización neta nula se organiza en dominios magnéticos. 

<video width="600" controls>
 <source src="T=1.0.N=10000.mp4" type="video/mp4">



 En este caso T=5.0, N=100 y los pasos montecarlo son 10000. La configuración inicial es ordenada y la magnetización total nula. 

 <video width="600" controls>
 <source src="T=5.0. N=10000.mp4" type="video/mp4">

# Incertidumbres
Para estimar la incertidumbre de las medidas obtenidas en la simulación, es fundamental calcular la varianza de la magnitud de interés . La varianza mide la dispersión de los valores respecto a su media y se calcula como:

$$ \sigma^2 \approx \frac{1}{N} \sum_{i=1}^{N} (x_i - \bar{x})^2 $$

donde $x_i$ representa cada una de las medidas individuales, $\bar{x}$ es el valor medio de la muestra y $N$ es el número total de medidas. 

El error estándar de la media (también llamado "1 sigma") se obtiene como:

$$
\Delta X = \frac{\sigma}{\sqrt{N}}
$$

donde $N$ es el número de medidas independientes. La mayoría de barras de error en las gráficas no se aprecian pues al considerar tamaños de la red $N$ relativamente grandes, estas disminuyen significativamente.

Se ha considerado el intervalo de 1 sigma. Es error representa el intervalo de confianza del 68% (aproximadamente) para una distribución normal, es decir, hay un 68% de probabilidad de que el valor real esté dentro de $\langle X \rangle \pm \Delta X$. 

- **1 sigma**: Intervalo de confianza del 68,26% ($\langle X \rangle \pm \Delta X$)
- **2 sigmas**: Intervalo de confianza del 95,44% ($\langle X \rangle \pm 2\Delta X$)
- **3 sigmas**: Intervalo de confianza del 99.74% ($\langle X \rangle \pm 3\Delta X$)




## Exponentes críticos

Los **exponentes críticos** son parámetros que describen cómo se comportan ciertas magnitudes físicas cerca de un punto de transición de fase, como la temperatura crítica $T_c$. Matemáticamente, indican cómo divergen o se anulan funciones termodinámicas (magnetización, calor específico, susceptibilidad, etc.) cuando la temperatura se aproxima a $T_c$, siguiendo leyes de potencias del tipo:

$$
F(T) \sim |T - T_c|^d
$$

donde  $d$ es el exponente crítico.

Los exponentes teóricos son:

| Función              | Magnetización $\beta$ | Calor $\alpha$ | Susceptibilidad  $\gamma$ |
|----------------------|--------------------------|-------------------|------------------------------|
| Exponente crítico    | $\beta = \frac{1}{8}$ | $\alpha = 0$    | $\gamma = \frac{-7}{4}$    |

donde el exponente correspondiente al calor específico es 0 debido a que presenta una divergencia de tipo logarítmica.

En el **modelo de Ising** bidimensional, los exponentes críticos más relevantes son:

- **$\beta$ (magnetización):** Describe cómo la magnetización espontánea desaparece al acercarse a $T_c$ por debajo de la transición:
  $$
  M(T) \sim (T_c - T)^{\beta} \quad \text{para } T \lesssim T_c
  $$
- **$\gamma$ (susceptibilidad):** 
  $$
  \chi(T) \sim |T - T_c|^{-\gamma}
  $$
- **$\alpha$ (calor específico):**
  $$
  C_v(T) \sim |T - T_c|^{-\alpha}
  $$


Para determinar el exponente crítico $\beta$  a partir de los valores de la magnetización en función de la temperatura, se toma logaritmos a ambos lados de la expresión anterior. 
   $$
   \ln M(T) = \beta \ln (T_c - T) + \text{constante}
   $$
   Esto implica que si representamos $\ln M$ frente a $\ln (T_c - T)$, los puntos deberían alinearse sobre una recta de pendiente $\beta$.


La siguiente gráfica muestra la recta de ajuste experimental.

![expcrit](lnM_vs_lnT.png)

La pendiente obtenida es $-8.71\pm2.34$. No coincide de ningún modo con el valor teórico de $\beta$=1/8=0.125 y los puntos no se ajusta con demasiada fiabilidad a una recta (el coeficiente de pearson es 0.47).

 Los valores de la magnetización obtenidos no se alejan demasiado de lo predicho por la teoría, por lo que debería haber una fuente de error desconocida que no hemos considerado. 
 
 El valor de la temperatura crítica que se ha escogido para calcular la pendiente es $T_c=2.04$, que es la temperatura que se espera obtener para un sistema finito con N=128.  
 Se ha escogido el valor más grande de N y únicamente los puntos en un entorno cerca de la transición de fase, donde la expresión del $c_v$ en función de los exponentes críticos es válida. 



# Bibliografía
- Una revisión del modelo de Ising y su aplicación en sociología a través del modelo de Sznajd.  
  https://addi.ehu.es/bitstream/handle/10810/30543/TFG_Garcia_Berdote_Asier.pdf?sequence=1&isAllowed=y
- The Analytical Expressions for a Finite-Size 2D Ising Model.
M.Yu. Malsagov, I.M. Karandashev and B.V. Kryzhanovsky. https://arxiv.org/pdf/1706.02541
- Exponentes críticos. https://studylib.es/doc/6578222/los-exponentes-criticos


